In [ ]:
# import  Pytorch
import torch
from torch import nn
# import torchvision

import torchvision
from torchvision import datasets
from torchvision import transforms
from torchvision.transforms import ToTensor

import matplotlib.pyplot as plt

### The dataset we'll be using `CIFAR10`

In [ ]:
# setup train data

train_data = datasets.CIFAR10(
    root="data",
    train=True,
    download=True,
    transform=ToTensor(),
    target_transform=None
)
test_data = datasets.CIFAR10(
    root="data",
    train=False,
    download=True,
    transform=ToTensor(),
    target_transform=None
)

In [ ]:
len(train_data),len(test_data)

In [ ]:
image, label = train_data[0]
image,label

In [ ]:
classes_names = train_data.classes
classes_names

In [ ]:
class_to_idx = train_data.class_to_idx
class_to_idx

In [ ]:
train_data.targets

In [ ]:
image.shape , label

In [ ]:
image,label = train_data[0]
print(f"image shape: {image}")
image = image.permute(1,2,0)
plt.imshow(image.squeeze())
plt.title(label)

In [ ]:
# ploting more images
torch.manual_seed(42)
fig = plt.figure(figsize=(9,9))
rows, cols = 4,4
for i in range(1,rows* cols +1):
  random_idx = torch.randint(0,len(train_data), size=[1]).item()
  img,label = train_data[random_idx]
  img = img.permute(1,2,0)
  fig.add_subplot(rows,cols,i)
  plt.imshow(img.squeeze())
  plt.title(classes_names[label])
  plt.axis(False)

In [ ]:
train_data

## Preparing DataLoader

Right now , our data is in the form of 'Datasets'.

Using "DataLoader" will let us use the data as python iterable

In [ ]:
from torch.utils.data import DataLoader

#setting up the batch size hyperparameter
BATCH_SIZE = 32

# Turn datasets into Iterables
train_dataloader = DataLoader(dataset=train_data,
                              batch_size=BATCH_SIZE,
                              shuffle=True)
test_dataloader = DataLoader(dataset=test_data,
                             batch_size=BATCH_SIZE,
                             shuffle=False)

In [ ]:
# checking out the shape of training dataloader
train_features_batch , train_label_batch = next(iter(train_dataloader))
train_features_batch.shape,train_label_batch.shape

In [ ]:
# show a sample
torch.manual_seed(42)
random_idx = torch.randint(0, len(train_features_batch), size=[1]).item()
img,label = train_features_batch[random_idx], train_label_batch[random_idx]
img = img.permute(1,2,0)
plt.imshow(img.squeeze(), cmap="gray")
plt.title(classes_names[label])
plt.axis(False)
print(f"Image size: {img.shape}")
print(f"Label: {label}, label size: {label.shape}")

### Building a base-line model

In [ ]:
# creating a flatten layer
flatten_model = nn.Flatten()

# getting a single sample
x = train_features_batch[0]

# flatten the sample
output = flatten_model(x)

x.shape, output.shape

In [ ]:
# device agnostic code
device = "cuda" if torch.cuda.is_available() else "cpu"
device

In [ ]:
# using first linear model
from torch import nn
class CIFAR10MODEL_0(nn.Module):
  def __init__(self,
               input_shape: int,
               hidden_units: int,
               output_shape: int):
    super().__init__()
    self.layer_stack = nn.Sequential(
        nn.Flatten(),
        nn.Linear(in_features=input_shape, out_features=hidden_units),
        nn.Linear(in_features=hidden_units,out_features=output_shape)
    )
  def forward(self,x):
    return self.layer_stack(x)

# determining input parameters
model_0 = CIFAR10MODEL_0(
    input_shape=1024,
    hidden_units=10,
    output_shape=len(classes_names)
).to(device)
model_0

### Setup loss , optimizer & evalutation metrics

* loss function -> for Multi-Class classification , we use  `torch.nn.CrossEntropyLoss`
* Optimizer -> We can use SGD or Adam( for this project we are going to use `SGD`)
* Evaluation metrics -> Custom made accuracy funciton

In [ ]:
# loss function
loss_fn = nn.CrossEntropyLoss()

# Optimizer
optimizer = torch.optim.SGD(params= model_0.parameters(),
                            lr = 0.1)

# accuracy function
def accuracy_fn(y_true, y_pred):
  correct = torch.eq(y_true, y_pred).sum().item()
  acc = (correct/len(y_pred)) *100
  return acc

## Creating a training loop & training a model using batch data

In [ ]:
# training & testing

epochs = 3